# Лабораторная работа №5
## Выполнил студент группы Мжваия Н.Р БСТ2105

In [14]:
# Необходим при замере скорости выполнения кода
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict
import time
import pandas as pd

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [9]:
def KMP(text: str, pattern: str, ignore_сase=False) -> bool:
    if ignore_сase:
        text = text.lower()
        pattern = pattern.lower()
    t = len(text)
    p = len(pattern)
    if p > t:
        return False
    # массив содержащий максимальные длины суффиксов для каждого элемента в подстроке
    max_suffix = [0]*p
    j = 0
    for i in range(1, p-1):
        if pattern[j] == pattern[i]:
            max_suffix[i] = j+1
            j += 1
        else:
            if j == 0:
                max_suffix[i] = 0
            else:
                j = max_suffix[j-1]
    i, j = 0, 0 # i - итератор по тексту, j - итератор по подстроке
    while i < t:
        if text[i] == pattern[j]:
            i += 1
            j += 1
            if j == p:
                return True # подстрока найдена
        else:
            if j > 0:
                j = max_suffix[j-1]
            else:
                i += 1
    if i == t and j != p:
        return False  

In [10]:
pattern, options = input("Подстрока для поиска: "), bool(input("Ингорировать регистр (True/False): ").title())
with open("sources/0.txt", "r", encoding="utf-8") as f:
    data = f.read()
if KMP(data, pattern, options):
    print("Подстрока найдена")
else:
    print("Подстрока не найдена")

Подстрока для поиска: 0
Ингорировать регистр (True/False): 0
Подстрока не найдена


#### Упрощенный алгоритм Бойера-Мура

In [11]:
def BM(text: str, pattern: str, ignore_сase=False) -> bool:
    # значение по умолчанию для смещения, если буква отсутствует в паттерне
    def default_value():
        return len(pattern) 
    
    if ignore_сase:
        text = text.lower()
        pattern = pattern.lower()
    t = len(text)
    p = len(pattern)
    if p > t:
        return False
    d = defaultdict(default_value) # словарь смещений
    #заполнение словаря смещений
    for i in range(p-2, -1, -1):
        if pattern[i] not in d:
            d[pattern[i]] = p-i-1
    if pattern[p-1] not in d:
        d[pattern[p-1]] = p
    i = p-1 # счетчик проверяемого символа в тексте
    while(i < t):
        k = 0 # указатель внутри текста k = [0, len(text)-1], охватывает предыдущие символы в тексте
        j = 0 # указатель внутри паттерна
        mismatch = False
        for j in range(p-1, -1, -1): 
            if text[i-k] != pattern[j]:
                if j == p-1:
                    offset = d[text[i]] # смещение, если не равен последний символ образа
                else:
                    offset = d[pattern[j]] # смещение, если не равен не последний символ образа
                i += offset # смещение счетчика строки
                mismatch = True # если несовпадение символа, то mismatch = True
                break
            k += 1
        if not mismatch:
            return True
    return False

In [12]:
pattern, options = input("Подстрока для поиска: "), bool(input("Ингорировать регистр (True/False): ").title())
with open("sources/0.txt", "r", encoding="utf-8") as f:
    data = f.read()
if BM(data, pattern, options):
    print("Подстрока найдена")
else:
    print("Подстрока не найдена")

Подстрока для поиска: язык
Ингорировать регистр (True/False): 1
Подстрока найдена


In [15]:
texts = []
elems = []
for i in range(6):
    with open(f"sources/{i}.txt", "r", encoding="ansi") as f:
        texts.append(f.read())
KMP_time, BM_time, Python_time = [], [], []
for text in texts:
    elems.append(len(text))
    start = time.perf_counter()
    KMP(text, "простипрощайпривет")
    end = time.perf_counter()
    KMP_time.append(end-start)
    start = time.perf_counter()
    BM(text, "простипрощайпривет")
    end = time.perf_counter()
    BM_time.append(end-start)
    start = time.perf_counter()
    text.find("простипрощайпривет")
    end = time.perf_counter()
    Python_time.append(end-start)

In [16]:
df = pd.DataFrame({'Длина текста': [elems[i] for i in range(6)],
                   'Кнутт-Моррис-Пратт': [KMP_time[i] for i in range(6)],
                   'Бойер-Мур': [BM_time[i] for i in range(6)],
                   'Python': [Python_time[i] for i in range(6)]}).round(6)

df.set_index('Длина текста')

,Кнутт-Моррис-Пратт,Бойер-Мур,Python
Длина текста,,,
773,0.000129,0.000040,0.000002
348652,0.062193,0.014674,0.000192
384041,0.073218,0.016575,0.000201
445014,0.086740,0.020130,0.000243
1308910,0.237209,0.039186,0.000632
1527944,0.302216,0.068512,0.000836


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

### Вывод